## Lets look at the Data we can Fetch from Riot API

Now we need to fetch data on the user base that was generated from the previous RIOT games puuids. We will be looking into getting personal game details of each champion, to try and find what drives engagement especially on specific individuals over time.

First ensure packages are called, and create API call.

In [1]:
#pip install riotwatcher
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os 

# golbal variables
api_key = 'RGAPI-9bee7f5f-fe8c-4f40-8f1a-d18c4cd29126'
lol_watcher = LolWatcher(api_key)
my_region = 'EUW1'

## Import Data and Creating Functions

Through this part we will be importing the dataset that we previously created in the past code. We need the specific puuids to get match details. The dataframe will be iterated through and the puuid passed on to the api call.

In [229]:
##import dataset and convert puuids to dict.

puuids = pd.read_csv (r'folder/users/rankings_of_participants_combined.csv')

puuid_1 =  str(puuids.iat[1,2])
print(puuids)
#print(puuids.iat[1,0])
print (puuid_1)

        summonerName  summonerRanking  \
0         Nickel5870                5   
1     sasaki int acc                9   
2         Einhornisi               10   
3            El 1u1u               11   
4     JFF gamergrill               12   
...              ...              ...   
4688          HûaHuâ             1201   
4689    Kitty Nicole             1216   
4690      KC Rekkles             1218   
4691           Qnoxs             1348   
4692           Fiona             1420   

                                                  puuid  
0     HnublqKGARVGUs-E85zk8RjOQbnis_y0ht2LFP7skypFQ3...  
1     shjd66sWdNFO3-TrXGC1UShUpyZ0nBPJQ-Bbn59Is0oCpz...  
2     ldK32d-w-X1QMyY9WefsSb28_j8SKePOqPK2idqZQwLWPa...  
3     GL2AFV1KD-_ciurefUpullikNf8lHkIdD-k-iX7LHs4MT_...  
4     6oYOWYOVROHxCcwi10qbpOut6PBjWG86ZksEePfIYP4oEV...  
...                                                 ...  
4688  dIqO-wlwQQzLGsAgKLj4xKftKMAo-pyt_4hZAgPofmnKUJ...  
4689  ePO-fVMyQFdvCGjzIdDtJ_xLPSq2mL5gEGhRo

## Match List Function

This function takes in the region and the puuid desired from the function call. With these variables it calls the matchlist_by_puuid api call to get all of the players recent matches. Additionally we set count = 100 as this is the maximum amount the API call allows.

From that we create a for loop that creates a new df from with match_id and the player puuid that played in the game. The function call returns this dataframe with two columns.

In [2]:
#create a function to find the matches that recent puuids have been in. Return the list with matchids and puuids
def match_list(region, puuid):
    matches_for_player = lol_watcher.match.matchlist_by_puuid(region, puuid, count = 100)
    all_matches_puuid=[]
    new_matches = {}
    for i in matches_for_player:
        new_matches = {}
        new_matches['Matches'] = i
        new_matches['puuid']= puuid
        
        all_matches_puuid.append(new_matches)
        all_matches_puuid_df = pd.DataFrame(all_matches_puuid)
    return  all_matches_puuid_df

## Game Attributes Function

This function is designed to get all of the important information from the match details api call. 

The function takes in the region, match id and puuid and gets the match details of the single participant desired.

First it calls the match.by_id RIOT api call. From this we get the data dict and the data for the single desired participant needs to be flattened and extracted.

Next it uses a series of if statements for general match details. The ifs are required as some games do not have this information under the key 'info'.

Next we need to iterate through the dict key 'info' 'participants' to look for the participant desired (inputted puuid) to extract all match attributes that pertained to that player. Next if statement for matching puuid to dig into data of our puuid. There is a sub key of challenges and thus it is iterated through further to see which challenges were completed by the player in the specific match and flattened. Appened each attribute and create a row. The match detail for the desired player is returned flattened into a single row.



In [17]:
#this function will get all of the attributes of the game for the desired player we are looking at. It will then
#create a 1D row with all of the information to return.

def game_attributes(region, match_id, puuid):
    match_detail = lol_watcher.match.by_id(region,match_id)
    
    #just gets all the data from info
    participants_row = {}
    
    participants_row['matchID'] = match_id

    if 'gameCreation' in match_detail['info']:
        participants_row['gameCreation'] = match_detail['info']['gameCreation']
    if 'gameDuration' in match_detail['info']:
        participants_row['gameDuration'] = match_detail['info']['gameDuration']
    if 'gameStartTimestamp' in match_detail['info']:
        participants_row['gameStartTimestamp'] = match_detail['info']['gameStartTimestamp']
    if 'gameEndTimestamp' in match_detail['info']:
        participants_row['gameEndTimestamp'] = match_detail['info']['gameEndTimestamp']
    if 'gameMode' in match_detail['info']:
        participants_row['gameMode'] = match_detail['info']['gameMode']
    if 'gameType' in match_detail['info']:
        participants_row['gameType'] = match_detail['info']['gameType']
    if 'gameVersion' in match_detail['info']:
        participants_row['gameVersion'] = match_detail['info']['gameVersion']
    if 'mapId' in match_detail['info']:
        participants_row['mapId'] = match_detail['info']['mapId']

    #needs to iterate through participants as we only want our desired participant, end up only looking at data for
    #1 player
    for i in match_detail['info']['participants']:
        
        #only want our one player we have inputted.
        if(i['puuid'] == puuid ):
           
            #since there are sub keys in 'participants' such as perks and challenges, we need to iterate again
            #to get these values in 1D.
            for keys, values in i.items():
                
                #going further into dict of challenges to extract information
                if(keys== 'challenges'):
                    for keys_1, values_1 in i['challenges'].items():
                        
                        #create a label that says its a challenge
                        intermediate_challenge_key_name = keys_1 + "_challenge"
                        
                        #add the new column to row for challenges info
                        participants_row[intermediate_challenge_key_name] = values_1
                        
                    continue
                
                #skip keys
                if(keys== 'perks'):
                    #print('skipped perks')
                    continue
                
                #add the new column to row for participants
                participants_row[keys] = values
                
            
            
    return participants_row
        
    



## Creating a MatchID List for Puuid

This next block of code gets the matchlist from the match list function. It is done in this way instead of all one code to save all of the matchids, and not have too many api calls to riot in one go which is time consuming and prone to crashing.

In [273]:
#creating a list of matchids with their respective puuids

puuids_list = puuids.values.tolist()

all_player_matches = []

for champions in puuids_list:
    
    try:
        time.sleep(0.05)
        matches = match_list(my_region, champions[2])
        
        if(champions == puuids_list[0]):
            print('hello')
            dataframe = matches
        else:
            dataframe= pd.concat([dataframe, matches])
    
    except ApiError as err:
        if err.response.status_code == 503:
            #print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            continue
        if err.response.status_code == 403:
            #print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            continue
        else:
            continue
    
    
    

hello


ReadTimeout: HTTPSConnectionPool(host='europe.api.riotgames.com', port=443): Read timed out. (read timeout=None)

Save the dataframe to a file location to ensure data is saved.

In [287]:
#Match Id and participants puuid data list to csv
dataframe.to_csv('folder/users/matcheIDs_puuids.csv') 

Need to reread from the file we just saved to or skip this step if storing variables locally.

In [5]:
match_ids_puuids = pd.read_csv (r'folder/users/match_ids_puuids_u60.csv')


## Main

This part of the code will utilize the Game Attributes Function call to get the match details for each match.

The code iterates through the match list to go through all games by all players. It then calls the game attributes function to get the flattened row of match details for the desired player. It then creates the dataframe containing all matches for the desired Puuid list.

In [28]:
all_player_matches_stats = []

matches_by_player_list = match_ids_puuids.values.tolist()
i=0

#iterate through all the matches:
for match_id_1, puuid_1 in matches_by_player_list:
    #print(unique_match_id, '\n')
    i=i+1
    try:
        if(i % 1000==0):
          # Print on the screen some message
          print("iteration: ", i, "\n")
       
        
        
        if(i<15087):
            continue
        else:
            #print(match_id_1, puuid_1)
            #print(all_player_matches_stats)
            time.sleep(0.05)
            single_match_single_player_data = game_attributes(my_region, match_id_1, puuid_1 )
            
            single_match_single_player_data_df = pd.DataFrame(single_match_single_player_data, index=[0])

            #print(single_match_single_player_data_df)


            #if(match_id_1 == matches_by_player_list[0][0] and puuid_1==matches_by_player_list[0][1]):
            if(i==15087):
                all_player_matches_stats = single_match_single_player_data_df
            else:
                all_player_matches_stats= pd.concat([all_player_matches_stats, single_match_single_player_data_df])

        
        
    except ApiError as err:
        if err.response.status_code == 503:
            #print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            continue
        if err.response.status_code == 403:
            #print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            continue
        else:
            continue
            

#print(all_player_matches_stats)
all_player_matches_stats_pd = pd.DataFrame(all_player_matches_stats)

#print(all_player_matches_stats_pd)

iteration:  1000 

iteration:  2000 

iteration:  3000 

iteration:  4000 

iteration:  5000 

iteration:  6000 

iteration:  7000 

iteration:  8000 

iteration:  9000 

iteration:  10000 

iteration:  11000 

iteration:  12000 

iteration:  13000 

iteration:  14000 

iteration:  15000 

iteration:  16000 

iteration:  17000 

iteration:  18000 

iteration:  19000 

iteration:  20000 

iteration:  21000 

iteration:  22000 

iteration:  23000 

iteration:  24000 



In [29]:
#show dataframe
all_player_matches_stats

,matchID,gameCreation,gameDuration,gameStartTimestamp,gameEndTimestamp,gameMode,gameType,gameVersion,mapId,assists,...,killsOnLanersEarlyJungleAsJungler_challenge,teleportTakedowns_challenge,highestCrowdControlScore_challenge,thirdInhibitorDestroyedTime_challenge,fastestLegendary_challenge,mejaisFullStackInTime_challenge,takedownsFirstXMinutes_challenge,killingSprees_challenge,earliestElderDragon_challenge,mostWardsDestroyedOneSweeper_challenge
0,EUW1_5823023670,1649874895000,2317,1649874982278,1.649877e+12,CLASSIC,MATCHED_GAME,12.7.433.4138,11,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,EUW1_5822877870,1649872575000,1903,1649872727738,1.649875e+12,CLASSIC,MATCHED_GAME,12.7.433.4138,11,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,EUW1_5822920852,1649869735000,2329,1649869785292,1.649872e+12,CLASSIC,MATCHED_GAME,12.7.433.4138,11,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,EUW1_5822811517,1649865706000,1770,1649865737511,1.649868e+12,CLASSIC,MATCHED_GAME,12.7.433.4138,11,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,EUW1_5820636282,1649756890000,1476,1649756953846,1.649758e+12,CLASSIC,MATCHED_GAME,12.6.432.1258,11,9,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,EUW1_5901727001,1654119109000,2657,1654119133556,1.654122e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,26,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN
0,EUW1_5901680446,1654116868000,1744,1654116896315,1.654119e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,5,...,2.0,NaN,NaN,NaN,NaN,NaN,7.0,2.0,NaN,NaN
0,EUW1_5901643536,1654114537000,1799,1654114562665,1.654116e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,7,...,3.0,NaN,1.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN
0,EUW1_5901586309,1654112278000,1930,1654112305409,1.654114e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,12,...,1.0,NaN,NaN,NaN,NaN,NaN,7.0,1.0,1851.262524,NaN


In [31]:
#save data
all_player_matches_stats.to_csv('folder/users/u60_player_game_info_3.csv') 

In [43]:
all_1= pd.read_csv (r'folder/users/u60/u60_player_game_info_1.csv')

all_2= pd.read_csv (r'folder/users/u60/u60_player_game_info_2.csv')

all_3 = pd.read_csv (r'folder/users/u60/u60_player_game_info_3.csv')

In [45]:
frames = [all_1, all_2, all_3]

combined_all_player_matches_stats = pd.concat(frames)



In [48]:
combined_all_player_matches_stats

,matchID,gameCreation,gameDuration,gameStartTimestamp,gameEndTimestamp,gameMode,gameType,gameVersion,mapId,assists,...,highestCrowdControlScore_challenge,fastestLegendary_challenge,baronBuffGoldAdvantageOverThreshold_challenge,earliestBaron_challenge,earliestElderDragon_challenge,mejaisFullStackInTime_challenge,highestWardKills_challenge,controlWardTimeCoverageInRiverOrEnemyHalf_challenge,teleportTakedowns_challenge,mostWardsDestroyedOneSweeper_challenge
0,EUW1_5925108425,1655315067000,1138,1655315100456,1.655316e+12,CLASSIC,MATCHED_GAME,12.11.446.9344,11,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5925055432,1655313645000,1142,1655313732189,1.655315e+12,CLASSIC,MATCHED_GAME,12.11.446.9344,11,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5925091388,1655312069000,1264,1655312139015,1.655313e+12,CLASSIC,MATCHED_GAME,12.11.446.9344,11,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5803968309,1648848019000,1229,1648848112082,1.648849e+12,CLASSIC,MATCHED_GAME,12.6.431.7988,11,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5803943053,1648846292000,1000,1648846381983,1.648847e+12,CLASSIC,MATCHED_GAME,12.6.430.6775,11,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9221,EUW1_5901727001,1654119109000,2657,1654119133556,1.654122e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,26,...,NaN,NaN,1.0,1242.606171,NaN,NaN,NaN,NaN,NaN,NaN
9222,EUW1_5901680446,1654116868000,1744,1654116896315,1.654119e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9223,EUW1_5901643536,1654114537000,1799,1654114562665,1.654116e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,7,...,1.0,NaN,NaN,1288.367998,NaN,NaN,NaN,NaN,NaN,NaN
9224,EUW1_5901586309,1654112278000,1930,1654112305409,1.654114e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,12,...,NaN,NaN,1.0,1248.184452,1851.262524,NaN,1.0,NaN,NaN,NaN


In [47]:
combined_all_player_matches_stats = combined_all_player_matches_stats.drop('Unnamed: 0', 1)

In [49]:
combined_all_player_matches_stats.drop_duplicates()

,matchID,gameCreation,gameDuration,gameStartTimestamp,gameEndTimestamp,gameMode,gameType,gameVersion,mapId,assists,...,highestCrowdControlScore_challenge,fastestLegendary_challenge,baronBuffGoldAdvantageOverThreshold_challenge,earliestBaron_challenge,earliestElderDragon_challenge,mejaisFullStackInTime_challenge,highestWardKills_challenge,controlWardTimeCoverageInRiverOrEnemyHalf_challenge,teleportTakedowns_challenge,mostWardsDestroyedOneSweeper_challenge
0,EUW1_5925108425,1655315067000,1138,1655315100456,1.655316e+12,CLASSIC,MATCHED_GAME,12.11.446.9344,11,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5925055432,1655313645000,1142,1655313732189,1.655315e+12,CLASSIC,MATCHED_GAME,12.11.446.9344,11,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5925091388,1655312069000,1264,1655312139015,1.655313e+12,CLASSIC,MATCHED_GAME,12.11.446.9344,11,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5803968309,1648848019000,1229,1648848112082,1.648849e+12,CLASSIC,MATCHED_GAME,12.6.431.7988,11,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5803943053,1648846292000,1000,1648846381983,1.648847e+12,CLASSIC,MATCHED_GAME,12.6.430.6775,11,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9221,EUW1_5901727001,1654119109000,2657,1654119133556,1.654122e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,26,...,NaN,NaN,1.0,1242.606171,NaN,NaN,NaN,NaN,NaN,NaN
9222,EUW1_5901680446,1654116868000,1744,1654116896315,1.654119e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9223,EUW1_5901643536,1654114537000,1799,1654114562665,1.654116e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,7,...,1.0,NaN,NaN,1288.367998,NaN,NaN,NaN,NaN,NaN,NaN
9224,EUW1_5901586309,1654112278000,1930,1654112305409,1.654114e+12,CLASSIC,MATCHED_GAME,12.10.442.9993,11,12,...,NaN,NaN,1.0,1248.184452,1851.262524,NaN,1.0,NaN,NaN,NaN


In [51]:
#save data
combined_all_player_matches_stats.to_csv('folder/users/u60_player_game_info_combined_backup.csv') 